In [ ]:
# Notes from reading/studying Fluent Python by Luciano Ramalho
# Chapter 13
# Created 9/23/20

In [15]:
from array import array
import math
import numbers
import functools
from functools import reduce
import operator
from itertools import zip_longest
import string
import reprlib

In [21]:
# Final form after chap10

class Vector:
    typecode = 'd'
    shortcut_names='xyzt'
#     letters = string.ascii_lowercase

    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(self._components))
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
        
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
#         print(f'Index = {index}')
        cls = type(self)
        if isinstance(index, slice):
#             print("I'm a slice")
            x = cls(self._components[index])
#             print(x)
            return x
        elif isinstance(index, numbers.Integral):
#             print("I'm an Integral")
            return self._components[index]
        else:
            raise TypeError(f'Index must be integer or slice.\nYou selected: {index}')
    
    def __getattr__(self, attr):
        cls = type(self)
        attr = attr.lower()
        pos = cls.shortcut_names.find(attr)
        if 0 <= pos < len(cls.shortcut_names):
            return self._components[pos]
        else:
            raise AttributeError(f'Unknown attribute {attr} selected for {self!r}')
            
    def __setattr__(self, attr, value):
        cls = type(self)
        attr = attr.lower()
        pos = cls.shortcut_names.find(attr)
        if 0 <= pos < len(cls.shortcut_names):
            raise AttributeError(f'Attribute "{attr}" selected.  ' +
                                 f'"{cls.shortcut_names}" attributes are read only.')
        else:
            object.__setattr__(self, attr, value)
        
    def __hash__(self):
#         return self.hash1()
        return self.hash2()
    
    def hash1(self):
        hashes = [hash(i) for i in self._components]
#         x = reduce(lambda x, y: x^y, hashes)
        x = reduce(operator.xor, hashes, 0)
        return x

    def hash2(self):
        hashes = map(hash, self._components)
        x = reduce(operator.xor, hashes, 0)
        return x

    def __pos__(self):
        cls = type(self)
        return cls(self)
    
    def __neg__(self):
        cls = type(self)
        components = [-i for i in self._components]
        return cls(components)
    
    def __add__(self, other):
        cls = type(self)
        try:
            components = [i+j for i, j in zip_longest(self._components, other, fillvalue=0.0)]
        except TypeError:
            return NotImplemented
        return cls(components)

    def __radd__(self, other):
        return self + other

    def __eq__(self, other):
        if isinstance(other, Vector):
            if len(self._components) != len(other):
                return False
            return all(i==j for i, j in zip(self._components, other))
        else:
            return NotImplemented


In [23]:
b = Vector([1, 2])
c = Vector([3, 6])
d = b + (3.0, 6.0)
print(b, c, d)
e = (7.0, 11.0) + b
print(e)


(1.0, 2.0) (3.0, 6.0) (4.0, 8.0)
(8.0, 13.0)


In [17]:
b = Vector([1, 2])
c = +b
d = -b
print(b, c, d)
print(b is c, b == c)

(1.0, 2.0) (1.0, 2.0) (-1.0, -2.0)
False True


In [4]:
help(all)

Help on built-in function all in module builtins:

all(iterable, /)
    Return True if bool(x) is True for all values x in the iterable.
    
    If the iterable is empty, return True.



In [30]:
class my_mutable:
    def __init__(self):
        self.x = []
    def __iadd__(self, item):
        self.x.append(item)
        return self

In [31]:
y = my_mutable()
print(id(y), y, y.x)
y += 10
print(id(y), y, y.x)
y += 20
print(id(y), y, y.x)

102954416 <__main__.my_mutable object at 0x0622F5B0> []
102954416 <__main__.my_mutable object at 0x0622F5B0> [10]
102954416 <__main__.my_mutable object at 0x0622F5B0> [10, 20]
